In [1]:
import os

os.environ['KERAS_BACKEND'] = "torch"

In [2]:
from keras import layers
import torch as torch

class Average(layers.Layer):
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(Average, self).__init__(**kwargs)

    def call(self, x, mask=None):
        if mask is not None:
            mask = torch.cast(mask, torch.floatx())
            mask = torch.expand_dims(mask)
            x = x * mask
        return torch.sum(x, axis=-2) / torch.sum(mask, axis=-2)

    def get_output_shape_for(self, input_shape):
        return input_shape[0:-2] + input_shape[-1:]

    def compute_mask(self, x, mask=None):
        return None

In [7]:
import numpy as np

Average()(np.array([1., 2., 3., 4., 5.]))

RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
